[View in Colaboratory](https://colab.research.google.com/github/onexmaster/DS/blob/master/Facial.ipynb)

In [0]:
!rm fer2013.csv

In [5]:
!
wget https://drive.google.com/drive/u/0/folders/1_yl_Eg8bimuUsGZ6DU3dZjF_ud4iy3dG

SyntaxError: ignored

In [6]:
!wget https://drive.google.com/Face/fer2013.csv

--2018-10-25 19:33:11--  https://drive.google.com/Face/fer2013.csv
Resolving drive.google.com (drive.google.com)... 74.125.31.100, 74.125.31.139, 74.125.31.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.31.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: /start [following]
--2018-10-25 19:33:11--  https://drive.google.com/start
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.google.com/drive/ [following]
--2018-10-25 19:33:11--  https://www.google.com/drive/
Resolving www.google.com (www.google.com)... 74.125.141.147, 74.125.141.105, 74.125.141.106, ...
Connecting to www.google.com (www.google.com)|74.125.141.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘fer2013.csv’

fer2013.csv             [ <=>                ]  29.85K  --.-KB/s    in 0.001s  

2018-10-25 19:33:11 

In [7]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [0]:
#variables
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 256
epochs = 5

In [22]:
with open("fer2013.csv") as f:
    content = f.readlines()

lines = np.array(content)

num_of_instances = lines.size
print("number of instances: ",num_of_instances)
print("instance length: ",len(lines[1].split(",")[1].split(" ")))

number of instances:  35888
instance length:  2304


In [0]:
!rm fer2013.csv


In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
file_list = drive.ListFile({'q': "'1_yl_Eg8bimuUsGZ6DU3dZjF_ud4iy3dG' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: Facial.ipynb, id: 1GcKQAte7vA8JJOryIJtARiGWyjhi52Jy
title: fer2013.csv, id: 1jDX0sdtFnoaBit9ZAiXlS4hrqTA6Csys


In [0]:
fer2013 = drive.CreateFile({'id': '1jDX0sdtFnoaBit9ZAiXlS4hrqTA6Csys'})
fer2013.GetContentFile('fer2013.csv')

In [6]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
#variables
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 256
epochs = 5


In [8]:
with open("fer2013.csv") as f:
    content = f.readlines()

lines = np.array(content)

num_of_instances = lines.size
print("number of instances: ",num_of_instances)
print("instance length: ",len(lines[1].split(",")[1].split(" ")))

number of instances:  35888
instance length:  2304


In [0]:
#initialize trainset and test set
x_train, y_train, x_test, y_test = [], [], [], []

In [0]:
#transfer train and test set data
for i in range(1,num_of_instances):
    try:
        emotion, img, usage = lines[i].split(",")
          
        val = img.split(" ")
            
        pixels = np.array(val, 'float32')
        
        emotion = keras.utils.to_categorical(emotion, num_classes)
    
        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
    except:
    	print("",end="")


In [11]:
#data transformation for train and test sets
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

x_train /= 255 #normalize inputs between [0, 1]
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

28709 train samples
3589 test samples


In [0]:
#construct CNN structure
model = Sequential()

#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))

In [0]:
##batch process
gen = ImageDataGenerator()
train_generator = gen.flow(x_train, y_train, batch_size=batch_size)

In [0]:
model.compile(loss='categorical_crossentropy'
    , optimizer=keras.optimizers.Adam()
    , metrics=['accuracy']
)

In [48]:
fit = True

if fit == True:
	#model.fit_generator(x_train, y_train, epochs=epochs) #train for all trainset
	model.fit_generator(train_generator, steps_per_epoch=batch_size, epochs=epochs) #train for randomly selected one
else:
	model.load_weights('facial_expression_model_weights.h5') #load weights

Epoch 1/5
256/256 [==============================] - 17s 65ms/step - loss: 0.3412 - acc: 0.8759
Epoch 2/5
256/256 [==============================] - 17s 65ms/step - loss: 0.2897 - acc: 0.8945
Epoch 3/5
256/256 [==============================] - 17s 65ms/step - loss: 0.2663 - acc: 0.9037
Epoch 4/5
256/256 [==============================] - 17s 65ms/step - loss: 0.2500 - acc: 0.9082
Epoch 5/5
256/256 [==============================] - 17s 66ms/step - loss: 0.2132 - acc: 0.9230


In [49]:
#overall evaluation
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', 100*score[1])

3589/3589 [==============================] - 1s 211us/step
Test loss: 2.7989492402431457
Test accuracy: 56.004458067144114
